In [1]:
import pandas as pd
from nltk import FreqDist
import pickle
import numpy as np
from gensim.models import Word2Vec
from gensim.models import FastText

In [2]:
import numpy as np
import pandas as pd
import os
import requests
import re
from nltk import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from cltk.stops.grc import STOPS as stopwords
from gensim.corpora import Dictionary
import unicodedata
import json

In [3]:
LAGT = pd.read_parquet("../data/large_data/LAGT_theos.parquet")

In [4]:
LAGT.head(5)

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,lemmata_source,tlg_date,not_before,not_after,date_uncertain,tlg_epithet,provenience,lemmatacount,periods,cents
2,ogl0001,ogl0001.ogl001,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...",180,1Kgr,"[[Πινυτός, ἀντιγράφω, θαυμάζω, ἀποδέχω, Διονύσ...",grecy,None,101.0,200.0,None,[],christian,34,[roman_peak],[2CE]
8,tlg0005,tlg0005.tlg003,tlg0005.tlg003.1st1K-grc1.xml,Theocritus,Syrinx,Οὐδενὸς εὐνάτειρα Μακροπτολέμοιο δὲ μάτηρ μαί...,77,1Kgr,"[[οὐδενός, εὐνητήρ], [μακροπτολέμοιο, μήτηρ, μ...",grecy,4-3 B.C.,-400.0,-201.0,False,[Bucolici],pagan,61,"[classical, hellenistic]","[4BCE, 3BCE]"
9,tlg0006,tlg0006.tlg020,tlg0006.tlg020.1st1K-grc1.xml,Euripides,Fragmenta,ποίαν σε φῶμεν γαῖαν ἐκλελοιπότα πόλει ξενοῦσθ...,17708,1Kgr,"[[φημί, γῆ, ἐκλείπω, πόλις, ξενοῦσθαι], [πάτρα...",grecy,5 B.C.,-500.0,-401.0,False,[Tragici],pagan,10277,[classical],[5BCE]
10,tlg0007,tlg0007.tlg146,tlg0007.tlg146.1st1K-grc1.xml,Plutarch,Παροιμίαι αἷς Ἀλεξανδρεῖς ἐχρῶντο,Οἴκοι τὰ Μιλήσια: ἐπὶ τῶν ὅποι μὴ προςήκει τὴν...,2685,1Kgr,"[[Μιλήσιος], [προςήκω, τρυφή, ἐπιδείκνυμι], [Ἀ...",grecy,A.D. 1-2,1.0,200.0,False,"[Biographi, Philosophici/-ae]",pagan,1488,[roman_peak],"[1CE, 2CE]"
11,tlg0007,tlg0007.tlg147,tlg0007.tlg147.1st1K-grc1.xml,Plutarch,Ἐκλογὴ περὶ τῶν ἀδυνάτων,Κατὰ πετρῶν σπείρεις. Πλίνθον πλύνεις. Δικτύῳ ...,143,1Kgr,"[[πέτρα, σπείρω], [Πλίνθος, πλύνω, Δίκτυον, ἄν...",grecy,A.D. 1-2,1.0,200.0,False,"[Biographi, Philosophici/-ae]",pagan,125,[roman_peak],"[1CE, 2CE]"


In [5]:
religion_final = ["θεός", "Ζεύς", "εὐσεβής", 'ἱερός']
morality_final = ["ἀγαθός", "ἀρετή", "δίκαιος", "τιμή"]

with open("../data/vocabulary_balanced.pickle", "rb") as f:
    vocabulary_balanced = pickle.load(f)

In [6]:
wordcounts_df = pd.read_json("../data/large_data/wordcounts_df.json")
wordcounts_df.head(5)

,pagan_archaic,pagan_classical,pagan_hellenistic,pagan_roman_peak,pagan_roman_late,christian_roman_peak,christian_roman_late,jewish
εἰμί,2983.0,74864.0,31177.0,99272.0,19685.0,8324.0,5241.0,12319.0
πᾶς,1637.0,19928.0,9108.0,31800.0,15015.0,6883.0,9324.0,10776.0
ἀνήρ,1614.0,8550.0,3700.0,10259.0,4068.0,2053.0,3385.0,3086.0
αὐτός,1534.0,42498.0,25174.0,107013.0,28587.0,8447.0,9611.0,14221.0
θεός,1468.0,5595.0,2802.0,9629.0,6852.0,15349.0,20369.0,8856.0


In [7]:
wordcounts_df_balanced = wordcounts_df.loc[vocabulary_balanced]

# Read subcorpora from ngrams - developing a memory-friendly reader

In [8]:
with open("../data/subcorpora_ids_dict.pickle", "rb") as f:
    subcorpora_ids_dict= pickle.load(f)
    

In [9]:
ids_lines = pickle.load(open("../data/ids_lines_wide.pickle", "rb"))

In [10]:
list(ids_lines.items())[:5]

[('ogl0001.ogl001', (0, 25)),
 ('tlg0005.tlg003', (25, 70)),
 ('tlg0006.tlg020', (70, 5712)),
 ('tlg0007.tlg146', (5712, 6527)),
 ('tlg0007.tlg147', (6527, 6587))]

In [11]:
subcorpus_ids = subcorpora_ids_dict["pagan_archaic"]
len(subcorpus_ids)

59

In [12]:
lines_list = [ ]
for id in subcorpus_ids:
    lines_list.extend([l for l in range(ids_lines[id][0], ids_lines[id][1])])
ngrams_n = len(lines_list)
ngrams_n

126322

In [13]:
sample_size=1000000
if sample_size != None:
    sample_inx =np.random.randint(0, ngrams_n, sample_size)
else:
    sample_inx =[n for n in range(0, ngrams_n)]

In [14]:
sample_lines = np.array(lines_list)[sample_inx]
len(sample_lines)

1000000

In [15]:
sample_lines[:5]

array([6502150, 6236315, 6259082, 7442291, 7434404])

In [16]:
test_ngrams = []
with open("../data/large_data/corpus_ngrams_bydocid_wide.txt", "r") as f:
    lines = f.readlines()
    for li in sample_lines:
        line = lines[li]# for line index in subcorpus lineindexes
        if bool(line.split()):
            test_ngrams.append(line.split())

In [17]:
len(test_ngrams)

999246

# Sample corpora with memory-friendly reader

In [18]:
class NgramCorpusSample(object):
    def __init__(self, ids_list, ids_lines, fname, sample_size=None, sample_seed=1, bow=False, dct=None):
        self.ids_list = ids_list
        self.fname = fname
        self.bow  = bow
        self.dct = dct
        self.sample_size = sample_size
        self.ids_lines = ids_lines
        lines_list = [ ]
        for id in ids_list:
            lines_list.extend([l for l in range(ids_lines[id][0], ids_lines[id][1])])
        ngrams_n = len(lines_list)
        if sample_size != None:
            np.random.seed(sample_seed)
            sample_inx =np.random.randint(0, ngrams_n, sample_size)
        else:
            sample_inx =[n for n in range(0, ngrams_n)]
        sample_lines = np.array(lines_list)[sample_inx]
        self.len = len(sample_lines)
        self.sample_lines = sample_lines
    def __len__(self):
        #[el for el in self]
        return self.len
    def __iter__(self):
        with open(self.fname, "r") as f:
            lines = f.readlines()
            for li in self.sample_lines:
                line = lines[li] # for line index in subcorpus lineindexes
                if bool(line.split()):
                    if (self.bow) & (self.dct != None):
                        yield self.dct.doc2bow(line.split())
                    else:
                        yield line.split()

In [19]:
# test with classical

In [20]:
subcorpus_ids = subcorpora_ids_dict["pagan_classical"]
len(subcorpus_ids)

454

In [21]:
subcorpus_ngrams =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid_wide.txt", sample_size=1000000)

In [22]:
len(subcorpus_ngrams)

1000000

In [23]:
len([doc for doc in subcorpus_ngrams])

996432

In [24]:
FreqDist([w for ng in subcorpus_ngrams for w in ng]).most_common()[:50]

[('εἰμί', 204764),
 ('οὗτος', 157681),
 ('αὐτός', 125996),
 ('τις', 81863),
 ('λέγω', 73080),
 ('γίγνομαι', 68509),
 ('ἔχω', 67146),
 ('πολύς', 65793),
 ('ἄλλος', 56015),
 ('πᾶς', 55755),
 ('ποιέω', 43850),
 ('οὐδείς', 30993),
 ('πόλις', 29144),
 ('φημί', 27211),
 ('λόγος', 26680),
 ('δέω', 26351),
 ('τίς', 25856),
 ('ἀγαθός', 25741),
 ('μέγας', 25649),
 ('τοιοῦτος', 25200),
 ('ἀνήρ', 23337),
 ('ἐκεῖνος', 23317),
 ('δοκέω', 22008),
 ('ἄνθρωπος', 20137),
 ('λαμβάνω', 20125),
 ('ἕτερος', 18956),
 ('ἕκαστος', 18169),
 ('εἷς', 18117),
 ('πρῶτος', 17225),
 ('ὁράω', 16387),
 ('βούλομαι', 15338),
 ('ὅδε', 15326),
 ('σῶμα', 15220),
 ('θεός', 14806),
 ('χρόνος', 14480),
 ('ἅπας', 14277),
 ('δίδωμι', 13981),
 ('οἶδα', 13913),
 ('ἔρχομαι', 13687),
 ('φύσις', 13624),
 ('κακός', 13563),
 ('δύναμαι', 13430),
 ('ἀρχή', 13414),
 ('νόμος', 13281),
 ('Ἀθηναῖος', 13046),
 ('μηδείς', 12584),
 ('καλός', 12418),
 ('πράσσω', 12228),
 ('φαίνω', 12058),
 ('παῖς', 11988)]

In [25]:
#with open("../data/large_data/subcorpora_vocabs.pickle", "rb") as f:
#    subcorpora_vocabs = pickle.load(f)

In [26]:
wordcounts_df["christian_roman_peak"].sort_values(ascending=False)[:100]
#subcorpora_vocabs["christian"]["word_freqs"][:100]

λέγω         21028.0
οὗτος        15609.0
θεός         15349.0
γίγνομαι      8581.0
αὐτός         8447.0
              ...   
φέρω          1190.0
ζητέω         1190.0
ἀδελφός       1190.0
ἀπόστολος     1181.0
γνῶσις        1171.0
Name: christian_roman_peak, Length: 100, dtype: float64

In [27]:
subcorpus_ids = subcorpora_ids_dict["christian_roman_peak"]
subcorpus_ngrams_seed1 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid_wide.txt", sample_size=1000000, sample_seed=1)
dict(FreqDist([w for ng in subcorpus_ngrams_seed1 for w in ng]).most_common()[:100])

{'θεός': 98075,
 'λέγω': 92818,
 'οὗτος': 77341,
 'αὐτός': 61590,
 'εἰμί': 56022,
 'γίγνομαι': 53487,
 'λόγος': 52803,
 'πᾶς': 45122,
 'ἄνθρωπος': 43890,
 'ἔχω': 40480,
 'πολύς': 36891,
 'Ἰησοῦς': 34123,
 'ποιέω': 33791,
 'κύριος': 28679,
 'τίς': 27460,
 'φημί': 26234,
 'Χριστός': 25724,
 'ψυχή': 25139,
 'πατήρ': 23403,
 'δύναμαι': 23000,
 'ὁράω': 22534,
 'υἱός': 21696,
 'ἄλλος': 21367,
 'τις': 20061,
 'γῆ': 20052,
 'πνεῦμα': 19485,
 'μόνος': 18099,
 'σῶμα': 17698,
 'ἀγαθός': 16286,
 'λαμβάνω': 15970,
 'δίδωμι': 15771,
 'δύναμις': 15154,
 'ἀκούω': 14944,
 'οὐρανός': 14606,
 'ἔρχομαι': 14462,
 'Θεός': 14308,
 'νόμος': 13612,
 'μέγας': 13474,
 'οἶδα': 13359,
 'ἅγιος': 13235,
 'ὄνομα': 13152,
 'ἐκεῖνος': 12966,
 'ἡμέρα': 12646,
 'ἀνήρ': 12516,
 'βούλομαι': 12239,
 'πιστεύω': 12228,
 'οὐδείς': 12052,
 'προφήτης': 11722,
 'κόσμος': 11612,
 'ἀλήθεια': 11550,
 'ἔργον': 11285,
 'τοιοῦτος': 11216,
 'ὅλος': 11176,
 'πρῶτος': 10930,
 'ἕτερος': 10881,
 'φύσις': 10688,
 'ἕκαστος': 10552,
 'θεῖος': 

In [28]:
subcorpus_ngrams_seed1 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid_wide.txt", sample_size=1000000, sample_seed=1)
subcorpus_ngrams_seed2 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid_wide.txt", sample_size=1000000, sample_seed=2)

In [29]:
freqs_comparison = pd.DataFrame(
    [dict(wordcounts_df["christian_roman_peak"][:5000]),
    dict(FreqDist([w for ng in subcorpus_ngrams_seed1 for w in ng]).most_common()[:5000]),
    dict(FreqDist([w for ng in subcorpus_ngrams_seed2 for w in ng]).most_common()[:5000])]
).T

In [30]:
freqs_comparison.dropna().corr()

,0,1,2
0,1.000000,0.985305,0.985546
1,0.985305,1.000000,0.999940
2,0.985546,0.999940,1.000000


In [31]:
len(freqs_comparison)

8313

In [32]:
freqs_comparison[:4000].corr(method="spearman")

,0,1,2
0,1.000000,0.993027,0.993049
1,0.993027,1.000000,0.997279
2,0.993049,0.997279,1.000000


In [33]:
np.log2(freqs_comparison).corr(method="spearman")

,0,1,2
0,1.000000,0.992209,0.992057
1,0.992209,1.000000,0.988383
2,0.992057,0.988383,1.000000


In [34]:
# let's build two models based on the same subcorpus but sampled with different seed

In [35]:

dict(wordcounts_df_balanced["christian_roman_peak"])

{'ῥάχις': nan,
 'βροτός': 53.0,
 'συμπόσιον': 38.0,
 'ἠγερέθομαι': nan,
 'χέω': 21.0,
 'λιμός': 100.0,
 'ὕστερος': 291.0,
 'κλείω': 40.0,
 'κλύζω': nan,
 'στοχάζομαι': 22.0,
 'ἐπιστάτης': 19.0,
 'ξένη': 9.0,
 'ἐρῆμος': 54.0,
 'βίος': 1197.0,
 'ὀρθόω': 2.0,
 'ἑπόμενος': 8.0,
 'στρατηγέω': 6.0,
 'συκοφάντης': 32.0,
 'Ἀλκμήνη': 6.0,
 'ἀγωνίζομαι': 143.0,
 'βουκόλος': 1.0,
 'ἡνίοχος': 19.0,
 'ἀνάκειμαι': 107.0,
 'ἀπογράφω': 23.0,
 'ἀντικείμενος': 14.0,
 'εὐφυής': 13.0,
 'Τρωιός': nan,
 'καταδέω': 8.0,
 'καθεύδω': 67.0,
 'φλεγμαίνω': 13.0,
 'μάχαιρα': 156.0,
 'προσαυδάω': 1.0,
 'κατάσχεσις': 8.0,
 'σαίνω': 4.0,
 'Μωυσεύς': 117.0,
 'ἐξαιρετός': 71.0,
 'στεροπή': nan,
 'οὖρον': 1.0,
 'ἀφίστημι': 232.0,
 'τυτθός': nan,
 'ὑποτείνω': 1.0,
 'ποτήριον': 195.0,
 'παρθενία': 30.0,
 'δισχίλιοι': 2.0,
 'Μέδων': nan,
 'φυλάσσω': 496.0,
 'κακός': 1478.0,
 'συμβάλλω': 80.0,
 'ἀστήρ': 209.0,
 'ὁμωνυμία': 8.0,
 'ἧμαι': 3.0,
 'στηρίζω': 41.0,
 'Ῥόδος': 4.0,
 'Σικελία': 31.0,
 'φλαῦρος': 1.0,
 'ἄσπετος': nan

In [36]:
sub = "pagan_archaic"
wordcounts_df_balanced[sub].dropna()

ῥάχις           2.0
βροτός        265.0
συμπόσιον       3.0
ἠγερέθομαι     18.0
χέω           176.0
              ...  
ξανθός         54.0
συνέχω          4.0
κατορθόω        2.0
μάχη          194.0
ἐπιγινώσκω      2.0
Name: pagan_archaic, Length: 3875, dtype: float64

In [37]:
vocab_freqs = dict(wordcounts_df_balanced[sub].dropna())
len(vocab_freqs)

3875

# Model developement

In [38]:
%%time
sub = "pagan_archaic"

ids_lines = pickle.load(open("../data/ids_lines_wide.pickle", "rb"))


def from_sub_to_vectors(sub, sample_size=None, sample_seed=1):
    subcorpus_ids = subcorpora_ids_dict[sub]
    vocab_freqs = dict(wordcounts_df_balanced[sub].dropna())
    subcorpus_ngrams =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid_wide.txt", sample_size=sample_size, sample_seed=sample_seed)
    model = Word2Vec(vector_size=100, window=5, min_count=0, negative=5, ns_exponent=1, sg=0, epochs=5, workers=8)
    model.build_vocab_from_freq(word_freq=vocab_freqs)
    model.train(subcorpus_ngrams, total_examples=len(subcorpus_ngrams), epochs=model.epochs)
    return model.wv

CPU times: user 1.27 ms, sys: 1.45 ms, total: 2.72 ms
Wall time: 2.12 ms


In [124]:

parameters_list = [
    ("pagan_archaic", None, 1),
    ("pagan_archaic", None, 2),
    ("pagan_archaic", 500000, 1),
    ("pagan_archaic", 500000, 2),
    ("pagan_archaic", 1000000, 1),
    ("pagan_archaic", 1000000, 2),
    ("pagan_archaic", 2000000, 1),
    ("pagan_archaic", 2000000, 2),
]

vectors_dict = {}
for param_tup in parameters_list:
    vectors_dict[param_tup[0] + str(param_tup[1]) + "seed" + str(param_tup[2])] = from_sub_to_vectors(param_tup[0], param_tup[1], param_tup[2])

CPU times: user 4min 55s, sys: 44.6 s, total: 5min 40s
Wall time: 4min 14s


In [126]:
vectors_keys = list(vectors_dict.keys())

In [127]:
vectors_dict[vectors_keys[0]
].most_similar(religion_final[0])

[('ἀγήραος', 0.6468861103057861),
 ('θνητός', 0.5819391012191772),
 ('ζεύς', 0.5717981457710266),
 ('μάκαρ', 0.5703902840614319),
 ('φάσκω', 0.540467381477356),
 ('Ὀλύμπιος', 0.5273755192756653),
 ('ἀθάνατος', 0.5263208150863647),
 ('ὄλβιος', 0.5197359919548035),
 ('ἕδος', 0.5157771110534668),
 ('ὅμοιος', 0.5110285878181458)]

In [128]:
vectors_dict[vectors_keys[1]
].most_similar(religion_final[0])

[('ἀγήραος', 0.6451097130775452),
 ('ζεύς', 0.5735556483268738),
 ('τελευτάω', 0.5708635449409485),
 ('ἀθάνατος', 0.562354326248169),
 ('θνητός', 0.5494258403778076),
 ('Ζεύς', 0.5406737923622131),
 ('μάκαρ', 0.536141574382782),
 ('Κρονίδης', 0.5329270958900452),
 ('βροτός', 0.5275591015815735),
 ('ἕδος', 0.5273777842521667)]

In [131]:
dfs = []
for key in vectors_dict.keys():
    dfs.append(pd.DataFrame(vectors_dict[key].most_similar(religion_final[0], topn=len(vocab_freqs)), columns=['token', key]).set_index('token'))
merged_df = pd.concat(dfs, axis=1)

In [132]:
merged_df

,pagan_archaicNoneseed1,pagan_archaicNoneseed2,pagan_archaic500000seed1,pagan_archaic500000seed2,pagan_archaic1000000seed1,pagan_archaic1000000seed2,pagan_archaic2000000seed1,pagan_archaic2000000seed2
token,,,,,,,,
ἀγήραος,0.646886,0.645110,0.431869,0.446335,0.405841,0.363223,0.327474,0.335193
θνητός,0.581939,0.549426,0.357604,0.380123,0.353241,0.350329,0.354067,0.371062
ζεύς,0.571798,0.573556,0.206291,0.241334,0.160865,0.197162,0.175285,0.237514
μάκαρ,0.570390,0.536142,0.445982,0.372624,0.417776,0.378684,0.440751,0.456092
φάσκω,0.540467,0.523757,0.249002,0.272266,0.255894,0.214435,0.196442,0.237678
...,...,...,...,...,...,...,...,...
ἧμαι,-0.235856,-0.176308,-0.144547,-0.108756,-0.160108,-0.110747,-0.158499,-0.205553
ἕλκω,-0.237381,-0.221354,-0.227602,-0.197570,-0.289157,-0.298494,-0.295958,-0.305589
στενάχω,-0.239951,-0.192660,-0.163511,-0.295064,-0.217923,-0.211869,-0.247994,-0.229054


In [133]:
merged_df.corr()

,pagan_archaicNoneseed1,pagan_archaicNoneseed2,pagan_archaic500000seed1,pagan_archaic500000seed2,pagan_archaic1000000seed1,pagan_archaic1000000seed2,pagan_archaic2000000seed1,pagan_archaic2000000seed2
pagan_archaicNoneseed1,1.000000,0.956038,0.553825,0.536277,0.526924,0.524120,0.576215,0.573959
pagan_archaicNoneseed2,0.956038,1.000000,0.577332,0.566769,0.550086,0.548186,0.591471,0.588815
pagan_archaic500000seed1,0.553825,0.577332,1.000000,0.912053,0.916872,0.910151,0.884127,0.877894
pagan_archaic500000seed2,0.536277,0.566769,0.912053,1.000000,0.900545,0.924758,0.882283,0.891084
pagan_archaic1000000seed1,0.526924,0.550086,0.916872,0.900545,1.000000,0.937604,0.923771,0.921724
pagan_archaic1000000seed2,0.524120,0.548186,0.910151,0.924758,0.937604,1.000000,0.915920,0.937014
pagan_archaic2000000seed1,0.576215,0.591471,0.884127,0.882283,0.923771,0.915920,1.000000,0.936832
pagan_archaic2000000seed2,0.573959,0.588815,0.877894,0.891084,0.921724,0.937014,0.936832,1.000000


In [ ]:
parameters_list = [
    ("pagan_roman_peak", None, 1),
    ("pagan_roman_peak", None, 2),
    ("pagan_roman_peak", 500000, 1),
    ("pagan_roman_peak", 500000, 2),
    ("pagan_roman_peak", 1000000, 1),
    ("pagan_roman_peak", 1000000, 2),
    ("pagan_roman_peak", 2000000, 1),
    ("pagan_roman_peak", 2000000, 2),
]
vectors_dict = {}
for param_tup in parameters_list:
    vectors_dict[param_tup[0] + str(param_tup[1]) + "seed" + str(param_tup[2])] = from_sub_to_vectors(param_tup[0], param_tup[1], param_tup[2])

In [ ]:
dfs = []
for key in vectors_dict.keys():
    dfs.append(pd.DataFrame(vectors_dict[key].most_similar(religion_final[0], topn=len(vocab_freqs)), columns=['token', key]).set_index('token'))
merged_df = pd.concat(dfs, axis=1)
merged_df.corr()

In [ ]:
merged_df.to_csv("../data/w2v_christian_roman_peak_corr", index=False)

# Training the full models

In [ ]:
ids_lines = pickle.load(open("../data/ids_lines_wide.pickle", "rb"))

vectors_full_dict = {}
for sub in subcorpora_ids_dict.keys():
    print("generating model for " + sub)
    vectors_full_dict[sub] = from_sub_to_vectors(sub, None, 1)
    print(sub + " model ready")

In [140]:
for key in vectors_full_dict.keys():
    print(key, vectors_full_dict[key].most_similar(religion_final[0]))

pagan_archaic [('ἀγήραος', 0.6705738306045532), ('θνητός', 0.5867362022399902), ('τελευτάω', 0.574408233165741), ('φάσκω', 0.571843147277832), ('ἀθάνατος', 0.5616358518600464), ('ὅρκος', 0.5421302914619446), ('βροτός', 0.5359242558479309), ('ἕδος', 0.5346282124519348), ('εὐνάω', 0.5322130918502808), ('στυγέω', 0.5313319563865662)]
pagan_classical [('δαίμων', 0.5859773755073547), ('εὐσεβής', 0.551023006439209), ('Ἀπόλλων', 0.5408515334129333), ('τελετή', 0.5009570717811584), ('δαιμόνιον', 0.4783174693584442), ('θυσία', 0.47501036524772644), ('εὐχή', 0.47273126244544983), ('θεά', 0.4726860523223877), ('θεῖος', 0.4662834107875824), ('σωτήρ', 0.46541956067085266)]
pagan_hellenistic [('θεῖος', 0.5941888093948364), ('Ζεύς', 0.5695279836654663), ('μυστήριον', 0.5550289154052734), ('τελετή', 0.5343388915061951), ('ἀθάνατος', 0.5151388049125671), ('θυσία', 0.4791301488876343), ('θεά', 0.47766241431236267), ('δαίμων', 0.4642082154750824), ('ἀθανασία', 0.4511449933052063), ('δαιμόνιον', 0.4457128

In [141]:
with open("../data/large_data/vectors_full_dict.pickle", "wb") as f:
    pickle.dump(vectors_full_dict, f)

# Training the model based on samples

In [ ]:

vectors_sample2m_dict = {}
for sub in subcorpora_ids_dict.keys():
    print("generating model for " + sub)
    vectors_sample2m_dict[sub] = from_sub_to_vectors(sub, 2000000, 1)
    print(sub + " model ready")

In [ ]:
with open("../data/large_data/vectors_sample2m_dict.pickle", "wb") as f:
    pickle.dump(vectors_sample2m_dict, f)